# A Jupyter Notebook to Add an Admin Login Flow
This notebook outlines the steps to add an admin login alongside the existing student flow in a Node.js and React project.

## Update User Schema
Modify the User schema to include a 'role' field with 'student' as the default value. Ensure no breakage for existing users.

In [ ]:
const mongoose = require('mongoose');
const Schema = mongoose.Schema;

const userSchema = new Schema({
    name: {
        type: String,
        required: true
    },
    studentID: {
        type: String,
        required: true
    },
    email: {
        type: String,
        required: true
    },
    password: {
        type: String,
        required: true
    },
    phoneNumber: {
        type: Number,
        required: true
    },
    role: {
        type: String,
        enum: ['student', 'admin'],
        default: 'student'
    }
});

module.exports = mongoose.model("UserModel", userSchema);

## Insert Admin User in MongoDB
Demonstrate how to insert an admin user into the database using MongoDB shell or a Node.js seed script.

In [ ]:
// Node.js seed script to insert an admin user
const mongoose = require('mongoose');
const UserModel = require('./Model/UserModel');

mongoose.connect('mongodb://localhost:27017/lost-and-found', { useNewUrlParser: true, useUnifiedTopology: true })
    .then(() => {
        console.log('Connected to MongoDB');
        return UserModel.create({
            name: 'Admin User',
            studentID: 'N/A',
            email: 'admin@example.com',
            password: 'securepassword', // Ensure to hash passwords in production
            phoneNumber: 1234567890,
            role: 'admin'
        });
    })
    .then(() => {
        console.log('Admin user created successfully');
        mongoose.disconnect();
    })
    .catch(err => {
        console.error('Error creating admin user:', err);
        mongoose.disconnect();
    });

## Create Admin Login Endpoint
Add a new '/admin/login' endpoint in the backend that authenticates users with the 'admin' role.

In [ ]:
// Backend: Admin login endpoint
const express = require('express');
const router = express.Router();
const UserModel = require('../Model/UserModel');
const bcrypt = require('bcryptjs');
const jwt = require('jsonwebtoken');

router.post('/admin/login', async (req, res) => {
    const { email, password } = req.body;

    try {
        const user = await UserModel.findOne({ email, role: 'admin' });
        if (!user) {
            return res.status(404).json({ message: 'Admin not found' });
        }

        const isPasswordValid = await bcrypt.compare(password, user.password);
        if (!isPasswordValid) {
            return res.status(401).json({ message: 'Invalid credentials' });
        }

        const token = jwt.sign({ id: user._id, role: user.role }, 'your_jwt_secret', { expiresIn: '1h' });
        res.status(200).json({ token });
    } catch (err) {
        res.status(500).json({ message: 'Server error', error: err });
    }
});

module.exports = router;

## Frontend: Admin Login Component
Create a React component for admin login and integrate it into the React Router.

In [ ]:
// React: AdminLogin.js
import React, { useState } from 'react';
import axios from 'axios';

const AdminLogin = () => {
    const [email, setEmail] = useState('');
    const [password, setPassword] = useState('');
    const [error, setError] = useState('');

    const handleLogin = async (e) => {
        e.preventDefault();
        try {
            const response = await axios.post('/admin/login', { email, password });
            localStorage.setItem('adminToken', response.data.token);
            window.location.href = '/admin/dashboard';
        } catch (err) {
            setError('Invalid credentials');
        }
    };

    return (
        <div>
            <h2>Admin Login</h2>
            <form onSubmit={handleLogin}>
                <input
                    type="email"
                    placeholder="Email"
                    value={email}
                    onChange={(e) => setEmail(e.target.value)}
                />
                <input
                    type="password"
                    placeholder="Password"
                    value={password}
                    onChange={(e) => setPassword(e.target.value)}
                />
                <button type="submit">Login</button>
            </form>
            {error && <p>{error}</p>}
        </div>
    );
};

export default AdminLogin;

## Protect Admin Routes
Implement middleware to protect admin routes and demonstrate how to use it in the backend.

In [ ]:
// Middleware to protect admin routes
const jwt = require('jsonwebtoken');

const adminAuth = (req, res, next) => {
    const token = req.header('Authorization')?.replace('Bearer ', '');
    if (!token) {
        return res.status(401).json({ message: 'Access denied' });
    }

    try {
        const decoded = jwt.verify(token, 'your_jwt_secret');
        if (decoded.role !== 'admin') {
            return res.status(403).json({ message: 'Forbidden' });
        }
        req.user = decoded;
        next();
    } catch (err) {
        res.status(400).json({ message: 'Invalid token' });
    }
};

module.exports = adminAuth;

In [ ]:
// Example usage of adminAuth middleware
const express = require('express');
const router = express.Router();
const adminAuth = require('../Middleware/adminAuth');

router.get('/admin/dashboard', adminAuth, (req, res) => {
    res.status(200).json({ message: 'Welcome to the admin dashboard' });
});

module.exports = router;